In [2]:
import csv
import cv2
import mediapipe as mp
import numpy as np
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

### Capturing landmarks from video


In [36]:
cap=cv2.VideoCapture("attempt.mp4")
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as pose:
    while cap.isOpened():
        success,frame= cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          break
        frame = cv2.resize(frame, (720, 800))
        #convert image to rgb
        img=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable= False
        #detection
        results=pose.process(img)
    
        #convert to bgr for opencv
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # render detection
        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(56, 28, 74), thickness=2, circle_radius=2),
        mp_drawing.DrawingSpec(color=(200, 87, 201), thickness=2, circle_radius=2))
    
    
        cv2.imshow("Mediapipe Feed", img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

### saving landmarks to csv

In [6]:
#csv file
f=open('attempt.csv',mode='w', newline='')
csv_writer=csv.writer(f)

header = []
for i in range(33): 
    header += [f'x{i}', f'y{i}', f'z{i}', f'v{i}']
csv_writer.writerow(header)

cap=cv2.VideoCapture("reference.mp4")
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as pose:
    while cap.isOpened():
        success,frame= cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          break
        frame = cv2.resize(frame, (720, 800))
        #convert image to rgb
        img=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable= False
        #detection
        results=pose.process(img)
    
        #convert to bgr for opencv
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # render detection
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(56, 28, 74), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(200, 87, 201), thickness=2, circle_radius=2))
    
            row=[]
            for lm in results.pose_landmarks.landmark:
                row.extend([lm.x, lm.y, lm.z, lm.visibility])
            csv_writer.writerow(row)
    
        cv2.imshow("Mediapipe Feed", img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
f.close()
cv2.destroyAllWindows()

Ignoring empty camera frame.


## Approach 1 - Direct comparison using landmarks

In [2]:
def read_csv(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        data = []
        for row in reader:
            frame = []
            for i in range(0, len(row), 4):
                x, y, z, v = map(float, row[i:i+4])
                frame.append([x, y, z])
            data.append(frame)
    return np.array(data)  

def compare_frames(frame1, frame2):
    frame1 = np.array(frame1)
    frame2 = np.array(frame2)
    distances = np.linalg.norm(frame1 - frame2, axis=1)  
    return np.mean(distances)

# extract both files
ref_pose = read_csv('ref.csv')
user_pose = read_csv('attempt.csv')

# match frame lengths
min_len = min(len(ref_pose), len(user_pose))
ref_pose = ref_pose[:min_len]
user_pose = user_pose[:min_len]

# compare 
scores = []
for f1, f2 in zip(ref_pose, user_pose):
    score = compare_frames(f1, f2)
    scores.append(score)
average_score = np.mean(scores)

# percentage score
MAX_POSSIBLE_DISTANCE = np.sqrt(3)
similarity_percentage = 100 * (1 - (average_score / MAX_POSSIBLE_DISTANCE))
similarity_percentage = max(0, min(similarity_percentage, 100))

print(f"Similarity Score: {similarity_percentage:.2f}%")
 


FileNotFoundError: [Errno 2] No such file or directory: 'ref.csv'